In [1]:
from konlpy.tag import Okt
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import pprint
from sklearn.pipeline import Pipeline

In [2]:
df = pd.read_csv('C:\\Users\\Kyungbin Choi\\Desktop\\Crawling\\Browndust\\web_crawling\\pycharm_project\\review_crwl_RM.csv')
df.head()
print("shape of dataframe : {}".format(df.shape))

shape of dataframe : (11520, 4)


In [3]:
doc_to_mat = df['review_txt'].as_matrix()
doc_to_mat = list(doc_to_mat)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [62]:
okt = Okt()
pprint.pprint(okt.tagset)

{'Adjective': '형용사',
 'Adverb': '부사',
 'Alpha': '알파벳',
 'Conjunction': '접속사',
 'Determiner': '관형사',
 'Eomi': '어미',
 'Exclamation': '감탄사',
 'Foreign': '외국어, 한자 및 기타기호',
 'Hashtag': '트위터 해쉬태그',
 'Josa': '조사',
 'KoreanParticle': '(ex: ㅋㅋ)',
 'Noun': '명사',
 'Number': '숫자',
 'PreEomi': '선어말어미',
 'Punctuation': '구두점',
 'ScreenName': '트위터 아이디',
 'Suffix': '접미사',
 'Unknown': '미등록어',
 'Verb': '동사'}


In [63]:
def tokenizer(raw, pos=["Noun","Verb", "Adjective"], stopword=[]):
    return [
        word for word, tag in okt.pos(raw, 
            norm=True,   # normalize 그랰ㅋㅋ -> 그래ㅋㅋ
            stem=True    # stemming 바뀌나->바뀌다
            )
            if len(word) > 1 and tag in pos and word not in stopword
        ]

In [79]:
vectorizer = CountVectorizer(tokenizer=tokenizer, min_df=5, ngram_range=(1,2)) #tokenizer, 최소 출현문서 수, ngram 범위
doc_to_vec = vectorizer.fit_transform(doc_to_mat)
print('fit_transform, (sentence {}, feature {})'.format(doc_to_vec.shape[0], doc_to_vec.shape[1]))

Wall time: 25.9 s
fit_transform, (sentence 11520, feature 5407)


In [80]:
print(doc_to_vec.toarray())
features = vectorizer.get_feature_names()
#pprint.pprint(features)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [81]:
doc_to_vec_df = pd.DataFrame(doc_to_vec.toarray(), columns=features)
doc_to_vec_df.head()

,가격,가기,가까이,가깝다,가끔,가능,가능 하다,가능 한게임,가능하다,가능하다 해주다,...,흥미,희귀,희다,희다 대의,희망,히트,힘내다,힘드다,힘드다 게임,힘들다
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [82]:
#doc_to_vec_df.to_csv('doc2vec_test.csv', index = False)

In [83]:
max_value = doc_to_vec.max(axis=0).toarray().ravel()
print(len(max_value))
sort_by_count = max_value.argsort()
features_name = np.array(vectorizer.get_feature_names())

5407


In [84]:
tokencnt = doc_to_vec_df.sum(axis=0)
tokencnt = tokencnt.sort_values()
print("상위 빈도 token 50개 : \n {}".format(tokencnt[-50:]))

상위 빈도 token 50개 : 
 최악         223
사람         223
즐기다        224
아인         228
재미         232
오류         233
행성         234
결제         239
들다         240
엔씨         240
진짜         242
아이템        243
서버         246
재미있다       249
생각         252
현질 유도      253
쓰다         256
그렇다        269
게임 하다      273
하다 하다      274
하다 게임      275
팅기다        279
그냥         279
보다         290
접속 안되다     296
재밌다        298
다시         304
계속         312
만들다        314
받다         318
먹다         326
최고         348
나오다        368
같다         378
않다         380
해주다        447
유도         483
뜨다         490
이다         499
유저         503
아니다        619
리니지        674
있다         726
업데이트       738
접속         767
되다         775
현질         821
과금         836
없다        1016
좋다        1053
dtype: int64


## TF_IDF

In [85]:
tfidfvectorizer = TfidfVectorizer(tokenizer=tokenizer, min_df=2,sublinear_tf=True)    # tf값에 1+log(tf)를 적용하여 tf값이 무한정 커지는 것을 막음)
tfidfvector = tfidfvectorizer.fit_transform(doc_to_mat)
print('fit_transform, (sentence {}, feature {})'.format(tfidfvector.shape[0], tfidfvector.shape[1]))
print(tfidfvector.toarray())
features = tfidfvectorizer.get_feature_names()
print(features)

fit_transform, (sentence 11520, feature 3182)
[[0.         0.16885281 0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
['가가', '가게', '가격', '가관', '가기', '가까이', '가깝다', '가끔', '가넷', '가능', '가능성', '가능하다', '가다', '가득하다', '가라', '가루', '가르다', '가르치다', '가만두다', '가면', '가문', '가바', '가방', '가버리다', '가볍다', '가보', '가보다', '가안', '가야', '가왜', '가요', '가입', '가장', '가정', '가져가다', '가져오다', '가족', '가죽', '가즈', '가지', '가지다', '가짐', '가치', '가하다', '가해', '각반', '각성', '각오', '각인', '각종', '간다', '간단하다', '간만', '간이', '간절하다', '간직', '간편하다', '간혹', '갈다', '갈리다', '갈수', '갈수록', '갈아타다', '감당', '감동', '감사', '감사하다', '감성', '감수', '감회', '감흥', '감히', '갑갑하다', '갑부', '갑옷', '갑자기', '강도', '강력하다', '강

In [86]:
tfidf_vec_df = pd.DataFrame(tfidfvector.toarray(), columns=features)
tfidf_vec_df.head()

,가가,가게,가격,가관,가기,가까이,가깝다,가끔,가넷,가능,...,흥행,희귀,희다,희망,희망이,흰색,히트,힘내다,힘드다,힘들다
0,0.0,0.168853,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [87]:
tfidf_vec_df.to_csv('tfidf_test.csv',index=False)

In [88]:
max_tfidf = tfidf_vec_df.sum(axis=0).sort_values()
print("tf-idf 상위 50 token: \n{}".format(max_tfidf[-50:]))


tf-idf 상위 50 token: 
많다       64.533850
에러       65.506188
들어가다     66.618337
그렇다      68.363992
다시       68.401754
즐기다      68.872135
들다       69.130347
심하다      69.473116
엔씨       69.490360
말다       69.844369
오류       71.520555
추억       73.172158
돼다       73.414873
받다       73.556937
만들다      79.182327
계속       81.639789
도박       81.672230
팅기다      83.634707
않다       86.361549
나오다      86.920197
쓰레기      87.347517
쓰다       91.326397
그냥       92.556583
같다       93.717130
최악       95.579304
유저       97.944306
행성      103.652346
해주다     104.823021
재미      105.381722
먹다      107.961339
뜨다      125.956224
아니다     129.893403
이다      144.277094
재다      146.233865
재미있다    146.931306
재밌다     151.235637
되다      155.640503
유도      160.411403
있다      162.249974
리니지     175.417223
접속      198.700281
과금      216.981722
최고      221.793271
없다      230.082319
업데이트    230.932384
현질      269.304415
안되다     331.461351
좋다      572.571696
게임      634.374458
하다      665.868006
dtype: float64


## Keyword related document

In [117]:
srch=[t for t in tokenizer("사행성") if t in features]
print("keyword token" , srch)
srch_dtm_tfidf = np.asarray(tfidfvector.toarray())[:, [
    # vectorize.vocabulary_.get 는 특정 feature 가 dtm 에서 가지고 있는 index값을 리턴한다
    tfidfvectorizer.vocabulary_.get(i) for i in srch]]
temp_df = pd.DataFrame(srch_dtm_tfidf, columns=srch)
temp_df = temp_df[(temp_df != 0).all(axis=1)] #하나의 token tf-idf값이 0이 있으면 제외
search_score_tfidf = temp_df.sum(axis=1)
print('related documents : {}'.format(len(search_score_tfidf)))
for i,j in zip(search_score_tfidf.sort_values(ascending=False).index, search_score_tfidf.sort_values(ascending=False).values):
    print('{} / tf-idf score : {}'.format(doc_to_mat[i], j))


keyword token ['행성']
related documents : 225
사행성ㅜㅜ / tf-idf score : 1.0
너무 사행성.. / tf-idf score : 1.0
사행성 / tf-idf score : 1.0
사행성 / tf-idf score : 1.0
사행성극암 / tf-idf score : 1.0
사행성임 / tf-idf score : 1.0
사행성 / tf-idf score : 1.0
사행성 / tf-idf score : 1.0
사행성 / tf-idf score : 1.0
사행성 / tf-idf score : 1.0
사행성갑 / tf-idf score : 1.0
너무 사행성 / tf-idf score : 1.0
사행성 / tf-idf score : 1.0
사행성 / tf-idf score : 1.0
사행성겜이네요 / tf-idf score : 1.0
사행성 / tf-idf score : 1.0
사행성 갑 / tf-idf score : 1.0
사행성 게임이에요 / tf-idf score : 0.8909973519648954
사행성게임 / tf-idf score : 0.8909973519648954
사행성게임 / tf-idf score : 0.8909973519648954
사행성게임 / tf-idf score : 0.8909973519648954
사행성게임 / tf-idf score : 0.8909973519648954
더러웅 사행성게임 / tf-idf score : 0.8909973519648954
사행성게임 / tf-idf score : 0.8909973519648954
사행성 게임 / tf-idf score : 0.8909973519648954
사행성쩌는게임 / tf-idf score : 0.8909973519648954
사행성 게임 / tf-idf score : 0.8909973519648954
사행성게임 / tf-idf score : 0.8909973519648954
사행성 게임.. / tf-idf score : 0.89099735

In [116]:
srch=[t for t in tokenizer("사행성") if t in features]
print("keyword token" , srch)
srch_dtm_cnt = np.asarray(doc_to_vec.toarray())[:, [
    # vectorize.vocabulary_.get 는 특정 feature 가 dtm 에서 가지고 있는 index값을 리턴한다
    vectorizer.vocabulary_.get(i) for i in srch]]
temp_df = pd.DataFrame(srch_dtm_cnt, columns=srch)
temp_df = temp_df[(temp_df != 0).all(axis=1)] #하나의 token count 값이 0이 있으면 제외
search_score_cnt = temp_df.sum(axis=1)
print('related documents : {}'.format(len(search_score_cnt)))
for i,j in zip(search_score_cnt.sort_values(ascending=False).index, search_score_cnt.sort_values(ascending=False).values):    
    print('{} / token count : {}'.format(doc_to_mat[i], j))

keyword token ['행성']
related documents : 225
일기장 가지고 빠징코 하는거만 봐다 사행성이라고 느껴지고 영웅영빈 합성만 봐도 사행성 쩌는데 국감에 나와서 리니지는 사행성 없다는 말보고 정내미 떨어짐 / token count : 3
그냥 이게임에 이벤트는없다 생각하세요 1:1문의? 할필요없어요 어차피 매크로 답변 옛추억리니지? 그런거없어요 그냥 현질유도게임에 이벤트도없는 게임이라생각하시면되요 현질은 최소10억 이벤트가 있다 그럼 그냥 없는걸로 치세요 이벤트템 주는거 아예없어요 사행성이 아니다 차라리 사행성 게임을하세요 이게임은 그냥 사기치는 게임입니다 못믿으시겠다 직접해보고 당해보세요 그럼ㅋ 확율0.0001% 템먹을 자신있으시면요 / token count : 2
이거 할시간에 이거에 돈지를 시간에 일을해라. 일하면서 해도되 근대 정신망가지는건 너네 책임이야. 물론나도 망가짐ㅋㅋㅋㅋㅋㅋ사행성 지린다 진짜..사행성의 끝판왕이다 이건 라스베가스 축소판이라고 생각해라 다쓸때까지 못나온다ㅋㅋㅋㅋㅋㅋ ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ거의 1년 다돼가는대 아직 개인거래 안뚫림? ㅋㅋㅋㅋㅋ약속 안지키죠 역시 3대 n가문 / token count : 2
사행성 게임..법적으로 규제가 필요할정도로 심함.. 빨리 망하는게 많은 젊은이들 구제하는거라 생각합니다..사행성게임 규제가 절실합니다..이거 국민청원이라도 해서 뭔가 조치가 필요해요.. / token count : 2
추억의 게임이 아님 돈없이는 할수 없는게임 이벤이 다돈임 한달 100 200백은 우수운게임 사행성 중 최고의 사행성 호구 되기 싫으시면 하지마세여 . 이벤은 돈이요 . 컨테츠는 사골처럼 우려 팔팔 우려먹고 현질해서 어든 아이템 다음이벤트면 업그레이드 버전 개속 우려먹기 유져 거지만들기 프로젝터 / token count : 2
사행성으로 돈버는 게임사가 사행성으로 템마춘다고 무과금 유저는 언제든 작업장으로 영정시키는 게임. / token count : 2
모르는분들 많은거 같은데 이게임 처음시작